In [ ]:
import spynnaker8 as p
import time
from matplotlib import pyplot as plt
import numpy as np
import spynnaker8.spynnaker_plotting as pl
import spynnaker8.utilities.neo_convertor as convert
from pyNN.utility.plotting import Figure, Panel
import os
import csv
import openpyxl
import random
import itertools

In [ ]:
#Moving into the directory which contains the batch files
os.chdir("/home/jovyan/SpeechRecognition/Weights_Graph_and_testing/CSV_files") #Create a batch folder 
file_name = "all_csvs_1.xlsx"


list_to_append_1 = []

book = openpyxl.load_workbook(file_name)
sheet = book.active
row_count = sheet.max_row
col_count = sheet.max_column

for j in range(2,row_count+1):
    list_of_data = []
    for k in range(2,col_count+1):
        list_of_data.append(sheet.cell(row=j, column=k).value)
    list_to_append_1.append(list_of_data)
    



#Moving into the directory which contains the batch files
os.chdir("/home/jovyan/SpeechRecognition/Weights_Graph_and_testing/CSV_files") #Create a batch folder 
file_name = "all_csvs_0.xlsx"


list_to_append_0 = []

book = openpyxl.load_workbook(file_name)
sheet = book.active
row_count = sheet.max_row
col_count = sheet.max_column

for j in range(2,row_count+1):
    list_of_data = []
    for k in range(2,col_count+1):
        list_of_data.append(sheet.cell(row=j, column=k).value)
    list_to_append_0.append(list_of_data)

In [ ]:
print(list_to_append_0)

In [ ]:
print(len(list_to_append_0))

In [ ]:
print(list_to_append_1)

In [ ]:
print(len(list_to_append_1))

In [ ]:
training_list = []
training_label = []

for i in range(50):
    training_list.append(list_to_append_0[i])
    training_label.append(0)
    
for i in range(3):
    training_list.append(list_to_append_1[i])
    training_label.append(1)


In [ ]:
print(training_list)

In [ ]:
print(len(training_list))

In [ ]:
temp = list(zip(training_list, training_label)) 
random.shuffle(temp) 
training_list, training_label = zip(*temp) 

In [ ]:
#NETWORK FOR TRAINING 
list_of_weights = []

start_time = time.time()

TotalDuration = 1000.0

weights_list = []

#initialising random weights for the first iteration
for i in range(10):
    weights = []
    for j in range(200):
        weights.append(random.uniform(0.001,0.01))
    weights_list.append(weights)

list_of_weights.append(weights_list)    

# yet to take weights and feed it to the network for the next feature vector
for i,feature in enumerate(training_list):
    
    NumYCells = 200
    NumZCells = 1
    
    correct_digit = training_label[i]

    #defining conductance based Izhikevich model
    model_Izh = p.extra_models.Izhikevich_cond

    snr_a=0.02 #0.005
    snr_b=0.2 #0.32
    snr_c=-65 #-65
    snr_d=8 #2
    snr_v_init = 30 #-70
    snr_u_init = -13 #snr_b * snr_v_init

    tau_ampa = 6###excitatory synapse time constant
    tau_gabaa= 4### inhibitory synapse time constant
    E_ampa = 0.0
    E_gabaa = -80.0
    current_bias = -2.
    cell_params_input = {'a': snr_a, 'b': snr_b, 'c': snr_c, 'd': snr_d,
                       'v': snr_v_init, 'u': snr_u_init,
                       'tau_syn_E': tau_ampa, 'tau_syn_I': tau_gabaa,
                       'i_offset': feature,
                       'e_rev_E': E_ampa, 'e_rev_I': E_gabaa,
                       }
    cell_params_output_target = {'a': snr_a, 'b': snr_b, 'c': snr_c, 'd': snr_d,
                       'v': snr_v_init, 'u': snr_u_init,
                       'tau_syn_E': tau_ampa, 'tau_syn_I': tau_gabaa,
                       'i_offset': sum(feature)/len(feature),
                       'e_rev_E': E_ampa, 'e_rev_I': E_gabaa,
                       }
    cell_params_output = {'a': snr_a, 'b': snr_b, 'c': snr_c, 'd': snr_d,
                       'v': snr_v_init, 'u': snr_u_init,
                       'tau_syn_E': tau_ampa, 'tau_syn_I': tau_gabaa,
                       'i_offset': -3.0,
                       'e_rev_E': E_ampa, 'e_rev_I': E_gabaa,
                       }


    p.setup(timestep=0.1, min_delay=1.0, max_delay=1.0)
    
    z_population = []


    #input population of 200
    y_population = p.Population(NumYCells, model_Izh(**cell_params_input), label='RS_Izh_neuron_input')

    #output population of 10
    for j in range(10):
        if j==correct_digit:
            z_population.append(p.Population(NumZCells, model_Izh(**cell_params_output_target), label='RS_Izh_neuron_output_target'))
        else:
            z_population.append(p.Population(NumZCells, model_Izh(**cell_params_output), label='RS_Izh_neuron_output'))

    ee_connector = p.AllToAllConnector()


    stdp_projection = []

    for j in range(10):
        
        if j==correct_digit: #if correct digit
            #defining Hebbian learning rule 
            timing_rule = p.SpikePairRule(tau_plus=20.,tau_minus=20.,A_plus=0.02,A_minus=0.02)
        else:
            #defining Anti-Hebbian learning rule
            timing_rule = p.SpikePairRuleAntiHebbian(tau_plus=20.,tau_minus=20.,A_plus=0.02,A_minus=0.02)

        #defining the STDP learning rule with the required parameters
        stdp_model = p.STDPMechanism(timing_dependence=timing_rule,weight_dependence=p.AdditiveWeightDependence(w_min=0.001,w_max=0.01),weight=weights_list[j],delay=0.1) #MultiplicativeWeightDependence, add appropriate weight range: 0.001 to 0.01
        stdp_projection.append(p.Projection(y_population,z_population[j],ee_connector,synapse_type=stdp_model))

    #simulation and results
    y_population.record(['v', 'spikes'])
    z_population[0].record(['v', 'spikes'])
    z_population[1].record(['v', 'spikes'])
    z_population[2].record(['v', 'spikes'])
    z_population[3].record(['v', 'spikes'])
    z_population[4].record(['v', 'spikes'])
    z_population[5].record(['v', 'spikes'])
    z_population[6].record(['v', 'spikes'])
    z_population[7].record(['v', 'spikes'])
    z_population[8].record(['v', 'spikes'])
    z_population[9].record(['v', 'spikes'])

    p.run(TotalDuration)

    #redifine weights_list to get new values so that they can be used in the next iteration
    weights_list = []
    for i in range(len(stdp_projection)):
        print("Weights:{}".format(stdp_projection[i].get('weight', 'list')))
        weights = stdp_projection[i].get('weight', 'list')
        weight_list = []
        for j in range(len(weights)):
            weight_list.append(weights[j][2])
        weights_list.append(weight_list)
        
        
    list_of_weights.append(weights_list)
    
    
    pre_spikes = y_population.get_data('spikes')
    post_spikes = []
    for i in range(10):
        post_spikes.append(z_population[i].get_data('spikes'))
        
    # post_spikes = z_population[0].get_data('spikes')

#     Figure(Panel(pre_spikes.segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[0].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[1].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[2].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[3].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[4].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[5].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[6].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[7].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[8].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[9].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), title="Speech Classifier", annotations="simulated with {}".format(p.name())).save('SpeechClassification.png')

    # Figure(Panel(pre_spikes.segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)), Panel(post_spikes[0].segments[0].spiketrains, yticks=True, markersize=0.2, xlim=(0,TotalDuration)))

    p.end()  
    
    print("**************************************************")
    print(weights_list)
    print("**************************************************")


In [ ]:
import matplotlib.pyplot as plt

time = 0.0
list_of_timesteps = []
for i in range(len(training_list)):
    time+=1000.0
    list_of_timesteps.append(time)
    
synapse_0 = []
synapse_1 = []
synapse_2 = []
synapse_3 = []
synapse_4 = []
synapse_5 = []
synapse_6 = []
synapse_7 = []
synapse_8 = []
synapse_9 = []

for i in range(len(list_of_weights[1:])):
    for j in range(len(list_of_weights[i])):
        if j==0:
            synapse_0.append(list_of_weights[i][j])
        elif j==1:
            synapse_1.append(list_of_weights[i][j])
        elif j==2:
            synapse_2.append(list_of_weights[i][j])
        elif j==3:
            synapse_3.append(list_of_weights[i][j])
        elif j==4:
            synapse_4.append(list_of_weights[i][j])
        elif j==5:
            synapse_5.append(list_of_weights[i][j])
        elif j==6:
            synapse_6.append(list_of_weights[i][j])
        elif j==7:
            synapse_7.append(list_of_weights[i][j])
        elif j==8:
            synapse_8.append(list_of_weights[i][j])
        elif j==9:
            synapse_9.append(list_of_weights[i][j])    

In [ ]:
synapse_0_0 = []
for i in range(len(synapse_0)):
#     print(len(synapse_0[i]))
    synapse_0_0.append(synapse_0[i][199])
    
synapse_1_0 = []
for i in range(len(synapse_1)):
    synapse_1_0.append(synapse_1[i][0])
    
plt.xlabel('Simulation Time Steps')
plt.ylabel('Weights associated with 0th output neuron')
# plt.plot(list_of_timesteps, synapse_0_0, 'ro')
plt.plot(synapse_0,'ro')
plt.show()
plt.savefig('Time_vs_Weights_Synapse_0_0.png')

In [ ]:
len(post_spikes[2].segments[0].spiketrains[0])

# if post_spikes[0].segments[0].spiketrains[0][0]

In [ ]:
testing_list = []
testing_label = []

# testing_list.append(list_to_append_0[30])
# testing_label.append(0)

for i in range(21,29):
    testing_list.append(list_to_append_0[i])
    testing_label.append(0)
    
for i in range(6,8):
    testing_list.append(list_to_append_1[i])
    testing_label.append(1)

In [ ]:
total_num = len(testing_list)
num_correct = 0

# yet to take weights and feed it to the network for the next feature vector
for i,feature in enumerate(testing_list):
    
    NumYCells = 200
    NumZCells = 1
    
    correct_digit = testing_label[i]

    #defining conductance based Izhikevich model
    model_Izh = p.extra_models.Izhikevich_cond

    snr_a=0.02 #0.005
    snr_b=0.2 #0.32
    snr_c=-65 #-65
    snr_d=8 #2
    snr_v_init = 30 #-70
    snr_u_init = -13 #snr_b * snr_v_init

    tau_ampa = 6###excitatory synapse time constant
    tau_gabaa= 4### inhibitory synapse time constant
    E_ampa = 0.0
    E_gabaa = -80.0
    current_bias = -2.
    cell_params_input = {'a': snr_a, 'b': snr_b, 'c': snr_c, 'd': snr_d,
                       'v': snr_v_init, 'u': snr_u_init,
                       'tau_syn_E': tau_ampa, 'tau_syn_I': tau_gabaa,
                       'i_offset': feature,
                       'e_rev_E': E_ampa, 'e_rev_I': E_gabaa,
                       }
    cell_params_output = {'a': snr_a, 'b': snr_b, 'c': snr_c, 'd': snr_d,
                       'v': snr_v_init, 'u': snr_u_init,
                       'tau_syn_E': tau_ampa, 'tau_syn_I': tau_gabaa,
                       'i_offset': 0.0,
                       'e_rev_E': E_ampa, 'e_rev_I': E_gabaa,
                       }


    p.setup(timestep=0.1, min_delay=1.0, max_delay=1.0)
    
    z_population = []


    #input population of 200
    y_population = p.Population(NumYCells, model_Izh(**cell_params_input), label='RS_Izh_neuron_input')

    #output population of 10
    for j in range(10):
        z_population.append(p.Population(NumZCells, model_Izh(**cell_params_output), label='RS_Izh_neuron_output'))

    ee_connector = p.AllToAllConnector()

    for j in range(10):
        wiring = p.AllToAllConnector()
        static_synapse = p.StaticSynapse(weight=weights_list[j], delay=0.1)
        connections = p.Projection(y_population, z_population[i], wiring, static_synapse)
        

    #simulation and results
    y_population.record(['v', 'spikes'])
    z_population[0].record(['v', 'spikes'])
    z_population[1].record(['v', 'spikes'])
    z_population[2].record(['v', 'spikes'])
    z_population[3].record(['v', 'spikes'])
    z_population[4].record(['v', 'spikes'])
    z_population[5].record(['v', 'spikes'])
    z_population[6].record(['v', 'spikes'])
    z_population[7].record(['v', 'spikes'])
    z_population[8].record(['v', 'spikes'])
    z_population[9].record(['v', 'spikes'])

    p.run(TotalDuration)
    
    
    pre_spikes = y_population.get_data('spikes')
    post_spikes = []
    for i in range(10):
        post_spikes.append(z_population[i].get_data('spikes'))
    
    index = -1
    max_num = 0
    for j in range(10):
        print(len(post_spikes[j].segments[0].spiketrains[0]))
        if len(post_spikes[j].segments[0].spiketrains[0])>max_num:
            max_num = len(post_spikes[j].segments[0].spiketrains[0])
            index = j
            
    print("Actual digit:")
    print(correct_digit)
    print("Predicted digit:")
    print(index)
    
    if correct_digit == index:
        num_correct+=1
        
    p.end()  

print("Total number of samples:")
print(total_num)
print("Number of samples correctly predicted:")
print(num_correct)
print("Accuracy:")
print(num_correct/total_num)